This notebook goes through how to define **geometry**, apply **Euclidian transformations**, apply **modifications**, **create new geometry from the existing geometry**, and how to **visualize the results**.

# IMPORTS

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from spomso.cores.helper_functions import generate_grid, smarter_reshape, hard_binarization
from spomso.cores.geom import GenericGeometry
from spomso.cores.geom_2d import Rectangle

# PARAMETERS

In [ ]:
# size of the volume
co_size = 4, 4
# resolution of the volume
co_resolution = 400, 400

show = "FIELD" # BINARY, FIELD
show_midplane = True

# COORDINATE SYSTEM

Point cloud of coordinates is created with the **generate_grid** function, which takes the size of the coordinate system and the number of points in each direction (**co_resolution**).

In [ ]:
coor, co_res_new = generate_grid(co_size, co_resolution)

If there is an even number of points along any direction the function will automatically convert it to an odd number, therefore the origin is always included in the point cloud. The point cloud is a **numpy.ndarray** with a shape of **(D, N)**, where **D** is the number of dimensions and **N** is the number of data points.

# CREATE SDFs

Create a **rectangle** with side lengths **1** and **0.5**:

In [ ]:
rectangle = Rectangle(1, 0.5)

## LOCATION

**Move** the center of the geometry (**rectangle**) by the vector **(0.1, 1, 0)**:

In [ ]:
rectangle.move((0.1, 1, 0))

Since the **rectangle** is originally centered at zero the center of **rectangle** is moved to **(0.1, 1, 0**).

Geometry can be moved to a specific location with the following function (in this case to **(1, 1, 0)**):

In [ ]:
rectangle.set_location((1, 1, 0))

***set_location*** overrides all previous ***set_location*** and ***move*** calls.

**Move** the **rectangle** back to the origin:

In [ ]:
rectangle.move((-1, -1, 0))

## SCALE

**Rescale** the object by a factor of **2**, which means the **rectangle** will now have side lengths of **2** and **1**:

In [ ]:
rectangle.rescale(2)

**Rescale** the **rectangle** once again by a factor of **1.5**:

In [ ]:
rectangle.rescale(1.5)

The side lengths of the **rectangle** are ***original_side_lengths $\cdot$ 2 $\cdot$ 1.5***.

The **rescaling factor** can be set to a specific value with:

In [ ]:
rectangle.set_scale(1.5)

***set_scale*** overrides all previous ***rescale*** and ***set_scale*** calls, therefore the side lengths are now **1.5** and **0.75**.

**Rescale** the side lengths back to **1** and **0.5**:

In [ ]:
rectangle.rescale(2/3)

## ROTATION

**Rotate** the **rectangle** around the **z-axis** for **45°**

In [ ]:
rectangle.rotate(np.pi/4, (0, 0, 1))

The **rotate** method can also accept a **rotation matrix** as an input.

Set the **rotation** to a specific **angle** and **axis**. In this case for **30°** around the **z-axis**:

In [ ]:
rectangle.set_rotation(np.pi/6, (0, 0, 1))

***set_rotation*** overrides all previous ***rotate*** or ***set_rotation*** calls.

**Rotate** the **rectangle** back to its original orientation:

In [ ]:
rectangle.rotate(-np.pi/6, (0, 0, 1))

## SHOW TRANSFORMATIONS

To see the applied **Euclidian transformations** use:

In [ ]:
applied_transformations = rectangle.transformations
print("Euclidian Transformations:", applied_transformations)

To see the **location** of the geometry use:

In [ ]:
center = rectangle.center
print("Position of the rectangle:", center)

To see the **scaling factor** applied to the geometry use:

In [ ]:
scale = rectangle.scale
print("Scaling of the rectangle:", scale)

To see the **rotation matrix** applied to the geometry use:

In [ ]:
rotation_matrix = rectangle.rotation_matrix
print("Rotation matrix of the rectangle:", rotation_matrix)

## NOTE

The **Euclidean transformations** are always applied after **modifications**. For example:

Apply a **rotation** of **45°** around the **z-axis**:

In [ ]:
rectangle.rotate(np.pi/4, (0, 0, 1))

Apply **mirroring** on the **x-axis**.  The original **rectangle** will be located at **(1, 0, 0)** and its **mirror** image at (**-1, 0, 0)**:

In [ ]:
rectangle.mirror((-1, 0, 0), (1, 0, 0))

The **rotation** was called before the **mirror** modification but the **rotation** is applied to the geometry after the **mirroring**.

To apply the **modifications** after the transformations a new object has to be crated:

In [ ]:
rectangle = GenericGeometry(rectangle.propagate)

Apply the **mirror** operation once more, but this time along the **y-axis**:

In [ ]:
rectangle.mirror((0, -0.5, 0), (0, 0.5, 0))

In certain cases it does not matter if the **Euclidian transformations** are applied after the **modifications**. As a rule of thumb, this is true for **modifications** where the location of the geometry does not change, This is true for **modifications** such as **mirror**, **repetitions**, and **instancing** modifications.

## EVALUATE

Evaluate the SDF of the **rectangle** to create a signed distance field 2D map. This is done by calling the **create** method and passing in the point cloud of coordinates:

In [ ]:
rectangle_pattern = rectangle.create(coor)

# BINARIZATION

Convert the distance field to a binary voxel map, where 1 corresponds to the interior and 0 to the exterior of the geometry.

In [ ]:
if show_midplane:
    field = smarter_reshape(rectangle_pattern, co_resolution)
    if show=="BINARY":
        pattern_2d = hard_binarization(field, 0)

if show=="BINARY":
    pattern = hard_binarization(rectangle_pattern, 0)

# PLOT

In [ ]:
print("Drawing results...")
# Mid-plane cross-section plot
if show_midplane and show=="BINARY":
    fig, ax = plt.subplots(1,1, figsize=(8.25, 8.25))
    ax.imshow(pattern_2d[:, :].T,
              cmap="binary_r",
              extent=(-co_size[0]/2, co_size[0]/2,
                      -co_size[1]/2, co_size[1]/2),
              origin="lower"
              )
    ax.grid()

    fig.tight_layout()
    plt.show()

if show_midplane and show == "FIELD":
    fig, ax = plt.subplots(1, 1, figsize=(8.25, 8.25))
    print(field.shape)
    ax.imshow(field[:, :].T,
              cmap="binary_r",
              extent=(-co_size[0] / 2, co_size[0] / 2,
                      -co_size[1] / 2, co_size[1] / 2),
              origin="lower"
              )
    cs = ax.contour(coor[0].reshape(co_res_new[0], co_res_new[1]),
                    coor[1].reshape(co_res_new[0], co_res_new[1]),
                    field[:, :],
                    cmap="plasma_r")
    ax.clabel(cs, inline=True, fontsize=10)
    ax.grid()